In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from bs4 import BeautifulSoup

import urllib.request as req
from urllib.request import urlopen
from urllib.request import HTTPError, URLError
import urllib.parse
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='/Users/hoon/Library/Fonts/NanumSquareRegular.ttf').get_name()
plt.rc("font", family=font_name)
import matplotlib as mpl
import re
import time

## 1. 아이폰/갤럭시 검색 시 나오는 콘텐츠 댓글 수집
+ Apple iPhone
    + ITSub잇섭
    + 주연 ZUYONI
    + 디에디트 THE EDIT
+ Samsung Galaxy
    + ITsub잇섭
    + UNDERkg
    + 주연 ZUYONI   
+ 참고 : https://summaryday.tistory.com/34

In [15]:
# 아이폰 13 or 아이폰 13 프로 둘 중 뭘 살지 고민된다면? 언빡싱&자세한 차이점 비교 분석!
# ITSub잇섭
# 2681개 댓글
driver = webdriver.Chrome("/users/hoon/chromedriver")
driver.get("https://www.youtube.com/watch?v=hGjulX7scZ4")
driver.implicitly_wait(3) # 댓글 창이 뜰 때까지 기다려주는 역할
time.sleep(1.5)

driver.execute_script("window.scrollTo(0, 800)")
time.sleep(3)


# 댓글 페이지 최하단까지 스크롤하기
last_height = driver.execute_script("return document.documentElement.scrollHeight")

# 특별한 일이 없는 한 코드를 무한 반복을 하도록 만드는 조건문
# 아래 if break로 명령이 끝나야할 시기를 정해줬음
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

time.sleep(1.5)

# 다른 방법은 페이지다운 키를 계속 눌러주는 반복명령어 사용하는 것. 다만 느리고 마지막이 언제인지 모르기 때문에 반복문 숫자를 잘 정해줘야한다는 번거로움 있음
# e = driver.find_element_by_tag_name('body')
# for i in range(500):
#   e.send_keys(Keys.PAGE_DOWN)
#   time.sleep(1)

#팝업 닫기
try:
    driver.find_element_by_css_selector("#dismiss-button > a").click()
except:
    pass



#댓글들 데이터로 리스트에 저장해두기
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

id_list = soup.select("div#header-author > h3 > #author-text > span")
comment_list = soup.select("yt-formatted-string#content-text")

id_final = []
comment_final = []

for i in range(len(comment_list)):
    temp_id = id_list[i].text
    temp_id = temp_id.replace('\n', '')
    temp_id = temp_id.replace('\t', '')
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id)

    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment)

In [18]:
df = pd.DataFrame({'id' : id_final, 'comment' : comment_final})
df

,id,comment
0,염토,Xs쓰다가 13pro로 갈아타는데 사진을 많이 찍는 편이라 pro 예약했는데 역시 ...
1,꽃게탕,영상보고 xr에서 13으로 갈아탔습니다 ㅜ4일차 후기라치면.. 그래도 요번엔 기본 ...
2,작살Jaksal,잇섭님은 진짜 어려운말도 쉽게 설명해주고 가려운부분 잘 긁어줘서 너무 보기 편합니다
3,소록,갤럭시와 아이폰의120Hz 차이가 궁금했는데 어디에도 언급이 없어서 아쉬워했는데 역...
4,귤탱이,승승장구 하는 잇섭님 깔끔한 비교 적절한 추천 감사합니다 . 예민하지 않아서 일반써...
...,...,...
415,Jun,아이폰 3gs부터 써왔지만 이번 13보고는왠지모르게 이제는 겔럭시 z플립으로 갈아타...
416,장잘냥,120hz는 웹툰볼때 체감이 확오더러고요 스크롤 내릴때 매우 부드럽게 내려지더라고요
417,23:05,개인적으로 그냥 13씀.13프로 그리고 지금까지 아이폰프로들을 보면 기스에 취약한 ...
418,느낌표,전 13 프로 120hz 엄청 체감됐어요 “ㄹㅇ 개쩐다” 이러면서


In [19]:
df.to_csv('data/0430.csv', encoding='utf-8')

In [30]:
df = pd.read_csv('data/0430.csv', index_col=0)
df

,id,comment
0,염토,Xs쓰다가 13pro로 갈아타는데 사진을 많이 찍는 편이라 pro 예약했는데 역시 ...
1,꽃게탕,영상보고 xr에서 13으로 갈아탔습니다 ㅜ4일차 후기라치면.. 그래도 요번엔 기본 ...
2,작살Jaksal,잇섭님은 진짜 어려운말도 쉽게 설명해주고 가려운부분 잘 긁어줘서 너무 보기 편합니다
3,소록,갤럭시와 아이폰의120Hz 차이가 궁금했는데 어디에도 언급이 없어서 아쉬워했는데 역...
4,귤탱이,승승장구 하는 잇섭님 깔끔한 비교 적절한 추천 감사합니다 . 예민하지 않아서 일반써...
...,...,...
415,Jun,아이폰 3gs부터 써왔지만 이번 13보고는왠지모르게 이제는 겔럭시 z플립으로 갈아타...
416,장잘냥,120hz는 웹툰볼때 체감이 확오더러고요 스크롤 내릴때 매우 부드럽게 내려지더라고요
417,23:05,개인적으로 그냥 13씀.13프로 그리고 지금까지 아이폰프로들을 보면 기스에 취약한 ...
418,느낌표,전 13 프로 120hz 엄청 체감됐어요 “ㄹㅇ 개쩐다” 이러면서


In [12]:
def youtube_crawl(path, url):
    driver = webdriver.Chrome(path)
    driver.get(url)
    driver.implicitly_wait(3)
    time.sleep(3)

    driver.execute_script("window.scrollTo(0, 800)")
    time.sleep(3)


    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.5)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1.5)

    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass



    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    id_list = soup.select("div#header-author > h3 > #author-text > span")
    comment_list = soup.select("yt-formatted-string#content-text")

    id_final = []
    comment_final = []

    for i in range(len(comment_list)):
        temp_id = id_list[i].text
        temp_id = temp_id.replace('\n', '')
        temp_id = temp_id.replace('\t', '')
        temp_id = temp_id.replace('    ', '')
        id_final.append(temp_id)

        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        comment_final.append(temp_comment)

    return id_final, comment_final

In [16]:
# 써본 사람만 아는 아이폰SE 3세대 의외의 장단점! 찐 사용후기 들고왔어요🍎
# 주연 ZUYONI 
# 536개 댓글
path = "/users/hoon/chromedriver"
url = "https://www.youtube.com/watch?v=nz_WHqpl6G0&t=49s"

id_final, comment_final = youtube_crawl(path, url)

In [17]:
df1 = pd.DataFrame({'id' : id_final, 'comment' : comment_final})
df1

,id,comment
0,화성,"내가 샀을때 실제로 느낄 수 있는, 피부에 와닿는 장단점을 알려주시는게 주연님 장점..."
1,준찬,정말 이해 하기 쉽고 지루하지않은 재밌는 영상으로 나와서 너무 잘 봤어용 앞으로도 ...
2,율하,3년 4개월가량 쓴 아이폰 8이 말썽이라 13으로 넘어갈지 14로 넘어갈지 고민하던...
3,자나 ZANA,"세상에,,, 기본 카메라 쓰시는데도 이쁘게 나오시는 건 반칙입니다 주연님,,,,,"
4,하금하금,우리 언니 쿨톤인가보다.. 파란색 이렇게까지 잘어울리는 사람 처음 봄 그나저나 주연...
...,...,...
477,s,주둥이
478,EoS,폼팩터좀바꿔….
479,놈우현,se3 거지폰 아니노 ㅋㅋ 13시리즈 사라
480,tiltshift,주붕이


In [13]:
# 59만원짜리 아이폰SE 3세대 직접 써봤습니다, 살지말지 알려드림
# 디에디트 THE EDIT
# 663개 댓글
path = "/users/hoon/chromedriver"
url = 'https://www.youtube.com/watch?v=IcKb79_IsHw&t=58s'

id_third, comment_third = youtube_crawl(path, url)

In [15]:
df2 = pd.DataFrame({'id' : id_third, 'comment' : comment_third})
df2

,id,comment
0,배병언,GOS 사태로 오랜만에 아이폰으로 넘어왔는데 이제야 디에디트의 영상의 비율과 좌측 ...
1,김병한,H님의 의견에 백프로 공감합니다29만원 차이로 미니를 사면압도적인 카메라와 디스플레...
2,배배,"SE2 2년 사용했었는데 배터리,화면크기 말고는 정말 만족하는 제품이었음 휴대성 한..."
3,김내영,영상이 아이폰에 최적화 되어 있는 거 같아서 너무 보기 좋아요!! 레터박스도 없도 ...
4,UR West,아이폰 6s 6년째 사용중..배터리 30분쓰면 20%남음.. 뭘 갈아타도 드라마틱한...
...,...,...
483,J,ㅇ0ㅇ 1빠
484,칩사마,15:40 대가리
485,분열정신,트루댑스 카메라 자체가 비싼듯...전면카메라는 기본으로 들어가야 하지.. 적외선 카...
486,DY N,이 분이 잇섭인가요?


In [34]:
df['contents'] = 'ITSub'
df1['contents'] = 'ZUYONI'
df2['contents'] = 'THE EDIT'

In [35]:
df_final = pd.concat([df, df1, df2], axis=0)
df_final

,id,comment,contents
0,염토,Xs쓰다가 13pro로 갈아타는데 사진을 많이 찍는 편이라 pro 예약했는데 역시 ...,ITSub
1,꽃게탕,영상보고 xr에서 13으로 갈아탔습니다 ㅜ4일차 후기라치면.. 그래도 요번엔 기본 ...,ITSub
2,작살Jaksal,잇섭님은 진짜 어려운말도 쉽게 설명해주고 가려운부분 잘 긁어줘서 너무 보기 편합니다,ITSub
3,소록,갤럭시와 아이폰의120Hz 차이가 궁금했는데 어디에도 언급이 없어서 아쉬워했는데 역...,ITSub
4,귤탱이,승승장구 하는 잇섭님 깔끔한 비교 적절한 추천 감사합니다 . 예민하지 않아서 일반써...,ITSub
...,...,...,...
483,J,ㅇ0ㅇ 1빠,THE EDIT
484,칩사마,15:40 대가리,THE EDIT
485,분열정신,트루댑스 카메라 자체가 비싼듯...전면카메라는 기본으로 들어가야 하지.. 적외선 카...,THE EDIT
486,DY N,이 분이 잇섭인가요?,THE EDIT


In [38]:
df_final = df_final.reset_index()
del df_final['index']
df_final

,id,comment,contents
0,염토,Xs쓰다가 13pro로 갈아타는데 사진을 많이 찍는 편이라 pro 예약했는데 역시 ...,ITSub
1,꽃게탕,영상보고 xr에서 13으로 갈아탔습니다 ㅜ4일차 후기라치면.. 그래도 요번엔 기본 ...,ITSub
2,작살Jaksal,잇섭님은 진짜 어려운말도 쉽게 설명해주고 가려운부분 잘 긁어줘서 너무 보기 편합니다,ITSub
3,소록,갤럭시와 아이폰의120Hz 차이가 궁금했는데 어디에도 언급이 없어서 아쉬워했는데 역...,ITSub
4,귤탱이,승승장구 하는 잇섭님 깔끔한 비교 적절한 추천 감사합니다 . 예민하지 않아서 일반써...,ITSub
...,...,...,...
1385,J,ㅇ0ㅇ 1빠,THE EDIT
1386,칩사마,15:40 대가리,THE EDIT
1387,분열정신,트루댑스 카메라 자체가 비싼듯...전면카메라는 기본으로 들어가야 하지.. 적외선 카...,THE EDIT
1388,DY N,이 분이 잇섭인가요?,THE EDIT


In [39]:
# df_final은 csv 파일로 저장
df_final.to_csv('data/youtube_iphone.csv', encoding='utf-8')

In [40]:
# 유튜브 갤럭시s 22 관련 댓글도 똑같은 방법으로 확보하겠습니다.

path = "/users/hoon/chromedriver"
url_list = ['https://www.youtube.com/watch?v=If16qIGOJTo', 'https://www.youtube.com/watch?v=6dtGQmLp5E4', 'https://www.youtube.com/watch?v=oJp3W2z9wGo']
# 1 : ITSub // 2 : UNDERkg // 3. ZUYONI

galaxy_id1, galaxy_comment1 = youtube_crawl(path, url_list[0])
galaxy_id2, galaxy_comment2 = youtube_crawl(path, url_list[1])
galaxy_id3, galaxy_comment3 = youtube_crawl(path, url_list[2])

In [41]:
galaxy_df1 = pd.DataFrame({'id' : galaxy_id1, 'comment' : galaxy_comment1})
galaxy_df2 = pd.DataFrame({'id' : galaxy_id2, 'comment' : galaxy_comment2})
galaxy_df3 = pd.DataFrame({'id' : galaxy_id3, 'comment' : galaxy_comment3})

galaxy_df1['contents'] = 'ITSub'
galaxy_df2['contents'] = 'UNDERkg'
galaxy_df3['contents'] = 'ZUYONI'

galaxy_df_final = pd.concat([galaxy_df1, galaxy_df2, galaxy_df3], axis=0)
galaxy_df_final = galaxy_df_final.reset_index()
del galaxy_df_final['index']
galaxy_df_final

,id,comment,contents
0,ITSub잇섭,※ 본 영상 제작 전 삼성전자에 두 번 이상 문의 후 LTPO라고 확답을 받았고 재...,ITSub
1,홀리몰리과카몰리,아 진짜 디자인 미쳤어 너무 이쁘다 색도 이쁘게 잘 뽑았고 S21이랑 전체적으로는 ...,ITSub
2,RC Play Ground,울트라는 동영상 촬영시 8k 60프레임 지원인가요?4k고 60프레임이 최고인가요?,ITSub
3,아이스아메리카노,플립때도 그렇고 이번 s22도 그렇고 디자인이 젊은세대 감성에 가까워져서 좋네요ㅎㅎ...,ITSub
4,한슬,플랫디자인은 예전부터 갤럭시 쓰던 사람으로엣지 나오기전 플랫을 선호했어서 이번에 2...,ITSub
...,...,...,...
1773,영상은언능언능,국방그린 가즈아,ZUYONI
1774,Local_minima,손이 작으셔서 그런가....? 울트라가 왜이리 커보이지ㅋㅋㅋㅋ,ZUYONI
1775,나는성동구고산자로LSM이다,누님 긴장해서 그런지 삑사리자꾸나네요 ㅋㅋㅋㅋㅋ,ZUYONI
1776,김밥,주연테크 사장님이신가요?,ZUYONI


In [42]:
galaxy_df_final.to_csv('data/youtube_galaxy.csv', encoding='utf-8')

In [43]:
#### Problem
# 1. label이 없다. 긍정인지 부정인지 파악할 수 없다.
# 2. 크리에이터에 대한 댓글 있다. 1200개의 댓글 중 아이폰에 대한 댓글만 자동으로 추출하는 방법 없다.
# 3. 라벨을 얻을 수 있는 데이터로 훈련하고 유튜브 댓글을 분석하는 방식으로 진행
# -> 네이버 쇼핑?

## 2. 네이버 쇼핑 댓글에서 훈련용 데이터 얻기

- 별 다섯개는 만족 / 이 외엔 불만족으로 처리
- 만족과 불만족 데이터 숫자를 비슷하게 만들어줘야
- 4개, 3개, 2개, 1개, 0개인 데이터 수에 맞춰 5개 데이터를 줄여주는 방법으로 트레이닝셋을 만들자
- 페이지 접속 -> 스크롤 -> 이동 -> 스크롤 -> 이동 방식으로 진행되어야함

In [44]:
def naver_crawl(path, url):
    driver = webdriver.Chrome(path)
    driver.get(url)
    driver.implicitly_wait(3)
    time.sleep(3)

    driver.execute_script("window.scrollTo(0, 800)")
    time.sleep(3)


    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.5)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1.5)

    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass



    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    id_list = soup.select("div#header-author > h3 > #author-text > span")
    comment_list = soup.select("yt-formatted-string#content-text")

    id_final = []
    comment_final = []

    for i in range(len(comment_list)):
        temp_id = id_list[i].text
        temp_id = temp_id.replace('\n', '')
        temp_id = temp_id.replace('\t', '')
        temp_id = temp_id.replace('    ', '')
        id_final.append(temp_id)

        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        comment_final.append(temp_comment)

    return id_final, comment_final

True

In [ ]:
naver_url = [
    'https://search.shopping.naver.com/catalog/30927477618?query=%EA%B0%A4%EB%9F%AD%EC%8B%9Cs22&NaPm=ct%3Dl2ojtbuo%7Cci%3Db0f3792735bcb01990f1303302afe7c57dde10b0%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D77a4e3967b29fbf0ff9f6e3cf43c6b958a93a55f',
    'https://search.shopping.naver.com/catalog/30927999618?query=%EA%B0%A4%EB%9F%AD%EC%8B%9Cs22&NaPm=ct%3Dl2okg0zc%7Cci%3D4aa473ee1ca1baea97f643200e916ae46328c566%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Debe10d8e14ff443d047bfaf7e8974da5434f6310',
    'https://search.shopping.naver.com/catalog/30927334623?query=%EA%B0%A4%EB%9F%AD%EC%8B%9Cs22&NaPm=ct%3Dl2okgncw%7Cci%3D52305ebb7e51a5f3491d9d5a117222a451164425%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D89b6ef0b19bb554ddb3fb10aec119e563b846850'
]